##Prerequisites

In [1]:
import torch
torch.__version__

'2.6.0+cu124'

In [2]:
!pip install -U transformers sentence-transformers  datasets tqdm

In [3]:
!pip install faiss-cpu

In [4]:
import torch
import transformers
import faiss
import sentence_transformers

print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("FAISS:", faiss.__version__)
print("Sentence Transformers:", sentence_transformers.__version__)

Torch: 2.6.0+cu124
Transformers: 4.53.2
FAISS: 1.11.0
Sentence Transformers: 5.0.0


##Exploratory Data Analysis

In [5]:
import pandas as pd

df = pd.read_csv("data.csv")
df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [6]:
print(df['intent'].value_counts())

intent
contact_customer_service    1000
complaint                   1000
check_invoice               1000
switch_account              1000
edit_account                1000
contact_human_agent          999
check_payment_methods        999
delivery_period              999
newsletter_subscription      999
get_invoice                  999
payment_issue                999
registration_problems        999
cancel_order                 998
place_order                  998
track_refund                 998
change_order                 997
set_up_shipping_address      997
check_refund_policy          997
create_account               997
get_refund                   997
review                       997
delivery_options             995
delete_account               995
recover_password             995
track_order                  995
change_shipping_address      973
check_cancellation_fee       950
Name: count, dtype: int64


In [7]:
df.isna().sum()

,0
flags,0
instruction,0
category,0
intent,0
response,0


In [8]:
df = df.rename(columns={"instruction": "question", "response": "answer"})  # adapt as needed

In [9]:
df.columns

Index(['flags', 'question', 'category', 'intent', 'answer'], dtype='object')

In [10]:
df["question"] = df["question"].astype(str).str.strip()
df["answer"] = df["answer"].astype(str).str.strip()

In [11]:
print(df.shape)

(26872, 5)


In [12]:
df.head()

,flags,question,category,intent,answer
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


##Define Embeddings

In [13]:

from sentence_transformers import SentenceTransformer

# Load the embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
# Extract all the questions
questions = df["question"].tolist()

# Generate vector embeddings (768-d)
question_embeddings = embedding_model.encode(questions, show_progress_bar=True)

Batches:   0%|          | 0/840 [00:00<?, ?it/s]

##Define FAISS Database

In [15]:
import faiss
import numpy as np

# Convert to numpy array (float32 required by FAISS)
question_embeddings = np.array(question_embeddings).astype("float32")

# Create FAISS index
index = faiss.IndexFlatL2(question_embeddings.shape[1])  # L2 = Euclidean distance
index.add(question_embeddings)

print(f"FAISS index has {index.ntotal} vectors.")

FAISS index has 26872 vectors.


In [16]:
def retrieve_similar_tickets(query, top_k=3):
    # Step 1: Embed the query
    query_embedding = embedding_model.encode([query])
    query_embedding = np.array(query_embedding).astype("float32")

    # Step 2: Search FAISS
    distances, indices = index.search(query_embedding, top_k)

    # Step 3: Fetch the matching answers from the dataframe
    matched_questions = [df.iloc[idx]["question"] for idx in indices[0]]
    matched_answers = [df.iloc[idx]["answer"] for idx in indices[0]]

    return matched_questions, matched_answers

In [17]:
query = "I want to cancel my recent purchase"
questions, answers = retrieve_similar_tickets(query)

for i in range(len(questions)):
    print(f"Matched Q{i+1}: {questions[i]}")
    print(f"Matched A{i+1}: {answers[i]}")
    print("---")

Matched Q1: how to cancel the purchase i made
Matched A1: Assuredly! I understand that you want to cancel the purchase you made. Let me guide you through the process step by step: 

1. Log in to your {{Online Company Portal Info}} using your credentials.
2. Navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section, where you can find a list of your previous purchases.
3. Identify the specific purchase you want to cancel. You may need to search for it by order number or date.
4. Once you've located the purchase, look for an option labeled '{{Online Order Interaction}}' or something similar. Click on it.
5. Follow the instructions provided by the system to complete the cancellation process.

If you encounter any difficulties or have any questions along the way, our dedicated customer support team is here to assist you. You can reach us during {{Customer Support Hours}} at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website

In [18]:
!pip install -U  accelerate bitsandbytes einops

##Import Zephyr Model

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "HuggingFaceH4/zephyr-7b-beta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16,
    trust_remote_code=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00005-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [20]:
def generate_response_local(query, retrieved_answers):
    context = "\n\n".join(retrieved_answers)

    prompt = f"""<|system|>
    You are a helpful, empathetic customer support assistant who always answers in a kind and professional tone.
    <|user|>
    Customer's Question:{query}

    Relevant Past Support Responses:{context}

    Now answer the customer's query in a clear, friendly tone:
    <|assistant|>
    """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("<|assistant|>")[-1].strip()

In [21]:
def get_rag_response(query, top_k=3):
    # Retrieve similar tickets
    matched_questions, matched_answers = retrieve_similar_tickets(query, top_k=top_k)

    # Generate the final LLM response
    final_response = generate_response_local(query, matched_answers)

    return final_response

##Sample Test

In [23]:
query = "How do I track my recent order?"
print(get_rag_response(query))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Dear valued customer,

    Thank you for reaching out to us regarding the tracking of your recent order. I'm more than happy to assist you with this query.

    To help you track your order, please visit our website and navigate to the 'Order Tracking' section. Here, you can enter your order number, which is {{Order Number}}, and you'll be provided with the most up-to-date information regarding the status and location of your package.

    Alternatively, you can contact our customer support team at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website URL}}. Our team will be more than happy to assist you further and provide you with any additional information you may need.

    Please don't hesitate to reach out to us if you have any further questions or concerns. Our team is here to help you every step of the way, and we're committed to ensuring a seamless and satisfying experience for all of our customers.

    Best regards,

    [Your Name]
    [Your

##We did it!!